# Transformer LSTM

En tant que tel, nous allons utiliser la library transformers pour obtenir des transformers pré-entraînés et les utiliser comme couches d'embeddings. Nous allons geler le transformer et entraîner uniquement le reste du modèle qui apprend des représentations produites par le transformer. Dans ce cas, nous utiliserons un LSTM bidirectionnel multicouche.

## Préparer les données

In [1]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Le transformer a déjà été formé avec un vocabulaire spécifique, ce qui signifie que nous devons nous entraîner avec exactement le même vocabulaire et également symboliser nos données de la même manière que le transformer l'a fait lors de son apprentissage initial.

Le transformer a également été formé avec des tokens spéciaux pour marquer le début et la fin de la phrase. Ainsi qu'avec les tokens : pad et unk. 

In [2]:
len(tokenizer.vocab)

init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


Le modèle a été formé sur des séquences d'une longueur maximale définie qui vaut 512. 

In [3]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2] # car les tokens de début et de fin deront ajoutés
    return tokens

init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

from torchtext import data

TEXT = data.Field(batch_first = True, # le transformer s'attend à ce que la dimension du lot soit la première,
                  use_vocab = False, # nous avons déjà le vocabulaire de notre texte, fourni par le transformer
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

512
101 102 0 100


In [4]:
train_data, valid_data, test_data = data.TabularDataset.splits(
        path='./data/', train='train.csv',
        validation='valid.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

print(f'Taille des données train: {len(train_data)}')
print(f'Taille des données de validation: {len(valid_data)}')
print(f'Taille des données test: {len(test_data)}')
LABEL.build_vocab(train_data)

Taille des données train: 18163
Taille des données de validation: 2270
Taille des données test: 2271


In [5]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort = False)

## Construire le modèle

In [6]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Au lieu d'utiliser une couche embedding pour obtenir des embedding pour notre texte, nous utiliserons le modèle de transformer pré-entraîné. Ces embeddings seront ensuite introduites dans un LSTM pour produire une prédiction du sentiment de la phrase d'entrée. Nous obtenons la taille de la dimension d'embedding (appelée hidden_size) du transformer via son attribut de configuration. 

In [7]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()       
        self.bert = bert  
        embedding_dim = bert.config.to_dict()['hidden_size']     
        self.rnn = nn.LSTM(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)   
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
       
        hidden = hidden[0]
        #hidden = [n layers * n directions, batch size, emb dim]
  
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output


In [8]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model):,} paramètres à entraîner')

Le modèle a 113,160,961 paramètres à entraîner


Afin de figer les paramètres (et non de les entraîner), nous devons définir leur attribut requires_grad sur False. Pour ce faire, nous parcourons simplement tous les named_parameters de notre modèle et s'ils font partie du modèle de transformer de bert, nous définissons require_grad = False.

In [10]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,678,721 trainable parameters


On obtient ainsi un nombre de paramètres semblables à nos autres modèles.

## Entraînement

In [11]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

def recall(preds, y):
    '''
    Retourne le recall
    '''
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float()       
    
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    if (tp + fn) == 0:
        recall = torch.zeros(1)
        
    recall = tp / (tp + fn)
    return recall

def f1_loss(preds, y):
    '''
    Retourne le score F1
    '''  
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    
    if (tp + fn) == 0 or (tp + fp) == 0 or (recall + precision == 0):
        f1 = torch.zeros(1)
    else:
        f1 = 2* (precision*recall) / (precision + recall)
    
    return f1

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_rec = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        rec = recall(predictions, batch.label)
        f1 = f1_loss(predictions, batch.label)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_rec += rec.item()
        epoch_f1 += f1.item() 
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_rec / len(iterator), epoch_f1 / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_rec = 0
    epoch_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            rec = recall(predictions, batch.label)
            f1 = f1_loss(predictions, batch.label)
              
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_rec += rec.item()
            epoch_f1 += f1.item() 
            
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_rec / len(iterator), epoch_f1 / len(iterator)

In [12]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc, train_rec, train_f1 = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_rec, valid_f1 = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Recall: {train_rec*100:.2f}% | Train F1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. Recall: {valid_rec*100:.2f}%  | Val. F1: {valid_f1*100:.2f}%')


Epoch: 01 | Epoch Time: 6m 32s
	Train Loss: 0.521 | Train Acc: 74.04% | Train Recall: 40.50% | Train F1: 49.88%
	 Val. Loss: 0.472 |  Val. Acc: 76.39% | Val. Recall: 32.73%  | Val. F1: 47.64%
Epoch: 02 | Epoch Time: 8m 57s
	Train Loss: 0.439 | Train Acc: 79.18% | Train Recall: 45.16% | Train F1: 55.15%
	 Val. Loss: 0.298 |  Val. Acc: 88.23% | Val. Recall: 44.40%  | Val. F1: 58.77%
Epoch: 03 | Epoch Time: 6m 47s
	Train Loss: 0.290 | Train Acc: 88.00% | Train Recall: 44.84% | Train F1: 58.90%
	 Val. Loss: 0.241 |  Val. Acc: 90.36% | Val. Recall: 45.24%  | Val. F1: 59.79%
Epoch: 04 | Epoch Time: 6m 37s
	Train Loss: 0.252 | Train Acc: 90.10% | Train Recall: 44.76% | Train F1: 59.43%
	 Val. Loss: 0.244 |  Val. Acc: 90.79% | Val. Recall: 45.05%  | Val. F1: 59.79%
Epoch: 05 | Epoch Time: 6m 38s
	Train Loss: 0.235 | Train Acc: 90.52% | Train Recall: 44.76% | Train F1: 59.55%
	 Val. Loss: 0.229 |  Val. Acc: 91.23% | Val. Recall: 44.81%  | Val. F1: 59.78%
Epoch: 06 | Epoch Time: 6m 38s
	Train Lo

## Test

In [14]:
model.load_state_dict(torch.load('tut6-model.pt'))
test_loss, test_acc, test_rec, test_f1 = evaluate(model, test_iterator, criterion)


print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%| Test Recall: {test_rec*100:.2f}%  | Test F1: {test_f1*100:.2f}%')

def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

Test Loss: 0.256 | Test Acc: 90.31%| Test Recall: 47.00%  | Test F1: 61.73%


In [15]:
predict_sentiment(model, tokenizer, "This film is terrible")

0.9925274848937988

In [16]:
predict_sentiment(model, tokenizer, "This film is great")

0.008738343603909016

## Référence : 

 - https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb